In [1]:
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

In [2]:
from ecc import PrivateKey, S256Point
from helper import decode_base58, SIGHASH_ALL, h160_to_p2pkh_address, hash160
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx, TxFetcher
from urllib.request import Request, urlopen
import json

In [3]:
privkey_1 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","big"))
my_address_1 = privkey_1.point.address(testnet=True)
print(my_address_1)

mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8


In [4]:
privkey_2 = PrivateKey(int.from_bytes(b"Oscar Eduardo Serna Rosero","little"))
my_address_2 = privkey_2.point.address(testnet=True)
print(my_address_2)

mwuCRN8bMLWN9ETs29U8KseDkau72M9byF


# Let's try to create a multisignature address that needs both signatures from privkey_1 and privkey_2

In [5]:
m=2
op_m = 82
n=2
op_n=82
length_pubkey=33
pubkey1=privkey_1.point.sec()
pubkey2=privkey_2.point.sec()

In [6]:
pubkey1.hex()

'03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4'

In [7]:
pubkey2.hex()

'031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee'

In [8]:
#redeem_script = Script([op_m, length_pubkey, pubkey1, length_pubkey, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script = Script([op_m, pubkey1, pubkey2, op_n, 174])#174:OP_CHECKMULTISIG
redeem_script

OP_2 03e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e4 031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee OP_2 OP_CHECKMULTISIG

In [9]:
serialized_redeem = redeem_script.raw_serialize()
serialized_redeem.hex()

'522103e07f96e5ba598431c0c994493a4ae988c9854c171d5d4bb140db0a27a4c853e421031b63f964d8c65d1d1136dcfe5033dedea88c2d411934ea48c9708410be84e5ee52ae'

In [10]:
multi_sig_addr = h160_to_p2pkh_address(hash160(serialized_redeem), testnet=True)
multi_sig_addr

'my7YMFfcujKi8uPaSFEfULfNUdpHx3bXJw'

# Now let's try to send some BTCs to this address

In [11]:
from script import p2sh_script
# Let's build the tx:
tx_outs =[]
tx_ins =[]

change = int(0.01 * 100000000)# we are going to fix this later
amountTx= int(0.01 * 100000000)

#It will send 0.01 to the multisig address
output = TxOut(amountTx, p2sh_script(decode_base58(multi_sig_addr)))
print(output)
tx_outs.append(output)

change_output = TxOut(change, p2pkh_script(decode_base58(my_address_1)))
print(change_output)
tx_outs.append(change_output)

prev1_tx_id = "02971382a72fc23c087723ed35686fdac019ae75f8084b7c9f51a07c56d00951"
prev_tx1 = TxFetcher.fetch(prev1_tx_id, testnet=True)
print(prev_tx1)

def get_index(outs_list):
    for index,out in enumerate(outs_list):
        if out.script_pubkey.cmds[2] == decode_base58(my_address_1):
            return index
    return "output index not found"
    
prev1_index= get_index(prev_tx1.tx_outs)
#prev1_index=0
print(f"index 1: {prev1_index}")

tx_in1 = TxIn(bytes.fromhex(prev1_tx_id),prev1_index)
print(tx_in1)
tx_ins.append(tx_in1)

my_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_tx)

# sign the inputs in the transaction object using the private key
for tx_input in range(len(tx_ins)):
    print(my_tx.sign_input(tx_input, privkey_1))
# print the transaction's serialization in hex

#Let's calculate the fee and the change:
tx_size = len(my_tx.serialize().hex())
fee_per_byte = 5 # I changed from 2 to 5 after sending this transaction because it had really low appeal to miners.
fee = tx_size * fee_per_byte
print(f"fee: {fee}")

change = prev_tx1.tx_outs[prev1_index].amount - fee - amountTx
print(f"change {change}")
#Let's make sure that we are actually spending the exact amount of the UTXO
total_send=fee+amountTx+change
diff = prev_tx1.tx_outs[prev1_index].amount-total_send
print(f"total {total_send}, diff: {diff}")


#Let's modify the amount in the change_output:
tx_outs[-1].amount = change

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_tx.sign_input(tx_input, privkey_1))

print(my_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))




1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 02971382a72fc23c087723ed35686fdac019ae75f8084b7c9f51a07c56d00951
version: 1
tx_ins:
0581629ea49735dcdb06fd94e7b187bc0af71934119d466ab762df7665e00fef:1
fcb130e47a883b6bc3b2ac11af9cc0b3dde92f05a7110b68d91f6877d7b1e38e:0
tx_outs:
2594462:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 0
02971382a72fc23c087723ed35686fdac019ae75f8084b7c9f51a07c56d00951:0
tx: aad3e57da3c8cb3d7f5d52df5a7c2179085120381049bb3c1eabef0b052c035d
version: 1
tx_ins:
02971382a72fc23c087723ed35686fdac019ae75f8084b7c9f51a07c56d00951:0
tx_outs:
1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
True
fee: 2240
change 1592222
total 2594462, diff: 0
True
01000000015109d0567ca0519

In [12]:
my_tx.verify_input(0)

True

In [13]:
li = ["a","b","c"]
print(*li)

a b c


# Now let's try to spend the UTXO from the multi-sig address

In [14]:
reload(tx)
tx_outs =[]
tx_ins =[]

amountTx= int(0.01 * 100000000)# we'll fix this later after calculating the fee

#It will send back all the money to our address_1 minus the fee
output = TxOut(amountTx,  p2pkh_script(decode_base58(my_address_1)))
print(output)
tx_outs.append(output)


prev1_tx_id = "7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811"
prev_tx1 = TxFetcher.fetch(prev1_tx_id, testnet=True)
print(prev_tx1)

def get_index(outs_list):
    for index,out in enumerate(outs_list):
        if out.script_pubkey.cmds[2] == decode_base58(my_address_1):
            return index
    return "output index not found"
    
prev1_index= get_index(prev_tx1.tx_outs)
#prev1_index=0
print(f"index 1: {prev1_index}")

tx_in1 = TxIn(bytes.fromhex(prev1_tx_id),prev1_index)
print(tx_in1)
tx_ins.append(tx_in1)

my_multisig_tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
print(my_multisig_tx)



# sign the inputs in the transaction object using the private key
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input_multisig(tx_input, [privkey_1, privkey_2], redeem_script))#redeem_script from earlier
# print the transaction's serialization in hex

#Let's calculate the fee and the change:
tx_size = len(my_multisig_tx.serialize().hex())
fee_per_byte = 5 # I changed from 2 to 5 after sending this transaction because it had really low appeal to miners.
fee = tx_size * fee_per_byte 
print(f"fee: {fee}")

amountTx = prev_tx1.tx_outs[prev1_index].amount - fee 
print(f"amountTx : {amountTx}")
#Let's make sure that we are actually spending the exact amount of the UTXO
total_send=fee+amountTx
diff = prev_tx1.tx_outs[prev1_index].amount-total_send
print(f"total {total_send}, diff: {diff}")


#Let's modify the amount in the change_output:
tx_outs[-1].amount = amountTx

#we sign again because the hash changed
for tx_input in range(len(tx_ins)):
    print(my_multisig_tx.sign_input(tx_input, privkey_1))

print(my_multisig_tx.serialize().hex())

#for tx_input in range(len(tx_ins)):
 #   print(my_tx.verify_input(tx_input))




1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
tx: 7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811
version: 1
tx_ins:
02971382a72fc23c087723ed35686fdac019ae75f8084b7c9f51a07c56d00951:0
tx_outs:
1000000:OP_HASH160 c104b576f5436309587aefa3ddddd5c295b90480 OP_EQUAL
1593566:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
index 1: 1
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:1
tx: 46e40371b8391127e4ff05b288b9595c2bf172781864186e90cc1269ac15693c
version: 1
tx_ins:
7dc40bfc367d0188df26e0a7ab801284a215f9d3c67975433c821ca3d7927811:1
tx_outs:
1000000:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
locktime: 0
[0, b"\x03\xe0\x7f\x96\xe5\xbaY\x841\xc0\xc9\x94I:J\xe9\x88\xc9\x85L\x17\x1d]K\xb1@\xdb\n'\xa4\xc8S\xe4", b'\x03\x1bc\xf9d\xd8\xc6]\x1d\x116\xdc\xfeP3\xde\xde\xa8\x8c-A\x194\xeaH\xc9p\x84\x10\xbe\x84\xe5\xee', OP_2 03e0

TypeError: object supporting the buffer API required